# Universal Bank -WE04 

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import recall_score
from sklearn.tree import DecisionTreeRegressor
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# set random seed to ensure that results are repeatable
np.random.seed(1)


# Load data


In [3]:
ubank=pd.read_csv("UniversalBank.csv")

# Check the missing values by summing the total na's for each variable

In [4]:
ubank.isna().sum()

ID                    0
Age                   0
Experience            0
Income                0
ZIP Code              0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64

In [5]:
ubank['CD Account'].value_counts()

0    4698
1     302
Name: CD Account, dtype: int64

# Dropping least important columns

In [6]:
ubank.drop(ubank.columns[[0,4]], axis=1, inplace=True)

# Data split -train and test

In [7]:
train_df, test_df = train_test_split(ubank, test_size=0.3,random_state=1)
target = 'CD Account'
predictors = list(ubank.columns)
predictors.remove(target)

In [8]:

scaler = preprocessing.StandardScaler()
scaler.fit(train_df[predictors])


train_X = scaler.transform(train_df[predictors]) 
train_y = train_df[target] 

test_X = scaler.transform(test_df[predictors])
test_y = test_df[target] 

# Logistic Regression using RandomSearch and Grid Search


In [9]:
score_measure = "recall"
kfolds = 5

param_grid = {'C':[0.001,0.01,0.1,1,10], # C is the regulization strength
               'penalty':['l1', 'l2','elasticnet','none'],
              'solver':['saga','liblinear'],
              'max_iter': np.arange(500,1000)
    
    
}

lr = LogisticRegression()
rand_search = RandomizedSearchCV(estimator = lr, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(train_X,train_y)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestlr = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is 0.6708245243128964
... with parameters: {'solver': 'liblinear', 'penalty': 'l2', 'max_iter': 851, 'C': 0.1}


C:\Users\santo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
910 fits failed out of a total of 2500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
280 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\santo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\santo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\santo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 457, in _check_solver
    raise ValueError(


In [10]:
score_measure = "recall"
kfolds = 5
best_penality = rand_search.best_params_['penalty']
best_solver = rand_search.best_params_['solver']
min_regulization_strength=rand_search.best_params_['C']
min_iter = rand_search.best_params_['max_iter']

#Using the best parameters from the Random Search to use as range for the parameters to do the grid search
param_grid = {
    
    'C':np.arange(min_regulization_strength-1,min_regulization_strength+1), 
               'penalty':[best_penality],
              'solver':[best_solver],
              'max_iter': np.arange(min_iter-400,min_iter+400)
}

logreg =  LogisticRegression()
grid_search = GridSearchCV(estimator = logreg, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1, # n_jobs=-1 will utilize all available CPUs 
                return_train_score=True)

_ = grid_search.fit(train_X,train_y)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestlogreg = grid_search.best_estimator_

Fitting 5 folds for each of 1600 candidates, totalling 8000 fits
The best recall score is 0.6708245243128964
... with parameters: {'C': 0.09999999999999998, 'max_iter': 451, 'penalty': 'l2', 'solver': 'liblinear'}


C:\Users\santo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
4000 fits failed out of a total of 8000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4000 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\santo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\santo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1464, in fit
    raise ValueError("Penalty term must be positive; got (C=%r)" % self.C)
ValueError: Penalty term must be positive; got (C=-0.9)

  warnings.warn(some_fits_failed_message, FitFailedWarning)
C:\User

In [11]:
c_matrix = confusion_matrix(test_y, grid_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")
Recall_lr={TP/(TP+FN)}

Accuracy=0.9780000 Precision=1.0000000 Recall=0.6024096 F1=0.7518797


In [18]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})
performance = pd.concat([performance, pd.DataFrame({'model':"logistic using random & grid search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

# SVM

# svm with linear kernel


In [36]:
svm_lin_model = SVC(kernel="linear",probability=True)
_ = svm_lin_model.fit(train_X, np.ravel(train_y))

In [24]:
model_preds = svm_lin_model.predict(test_X)
c_matrix = confusion_matrix(test_y, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"lin svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,logistic using random & grid search,0.955333,0.586957,0.650602,0.617143
0,svm using Random & Grid search,0.955333,0.586957,0.650602,0.617143
0,svm using Random & Grid search,0.955333,0.586957,0.650602,0.617143
0,lin svm,0.978000,1.000000,0.602410,0.751880


# svm with rbf kernel


In [26]:
svm_rbf_model = SVC(kernel="rbf", C=10, gamma='scale',probability=True)
_ = svm_rbf_model.fit(train_X, np.ravel(train_y))

In [27]:
model_preds = svm_rbf_model.predict(test_X)
c_matrix = confusion_matrix(test_y, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rbf svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,logistic using random & grid search,0.955333,0.586957,0.650602,0.617143
0,svm using Random & Grid search,0.955333,0.586957,0.650602,0.617143
0,svm using Random & Grid search,0.955333,0.586957,0.650602,0.617143
0,lin svm,0.978000,1.000000,0.602410,0.751880
0,rbf svm,0.974667,0.909091,0.602410,0.724638


# svm with poly kernel


In [28]:
svm_poly_model = SVC(kernel="poly",probability=True, degree=3, coef0=1, C=10)
_ = svm_poly_model.fit(train_X, np.ravel(train_y))

In [29]:
model_preds = svm_poly_model.predict(test_X)
c_matrix = confusion_matrix(test_y, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"poly svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [30]:
performance

,model,Accuracy,Precision,Recall,F1
0,logistic using random & grid search,0.955333,0.586957,0.650602,0.617143
0,svm using Random & Grid search,0.955333,0.586957,0.650602,0.617143
0,svm using Random & Grid search,0.955333,0.586957,0.650602,0.617143
0,lin svm,0.978000,1.000000,0.602410,0.751880
0,rbf svm,0.974667,0.909091,0.602410,0.724638
0,poly svm,0.970000,0.806452,0.602410,0.689655


# random search in SVM

In [31]:

score_measure = "precision"
kfolds = 5

param_grid = {
    'C':[0.1,1,10,100],
    'gamma':[1,0.1,0.01,0.001],
    'kernel':['poly']
    
}

SVM_R_out = SVC()
rand_search = RandomizedSearchCV(estimator = SVM_R_out, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

C:\Users\santo\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 16 is smaller than n_iter=500. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 16 candidates, totalling 80 fits
The best precision score is 1.0
... with parameters: {'kernel': 'poly', 'gamma': 0.1, 'C': 0.1}


In [32]:
model_preds = rand_search.predict(test_X)
c_matrix = confusion_matrix(test_y, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rand_poly_svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [33]:
performance

,model,Accuracy,Precision,Recall,F1
0,logistic using random & grid search,0.955333,0.586957,0.650602,0.617143
0,svm using Random & Grid search,0.955333,0.586957,0.650602,0.617143
0,svm using Random & Grid search,0.955333,0.586957,0.650602,0.617143
0,lin svm,0.978000,1.000000,0.602410,0.751880
0,rbf svm,0.974667,0.909091,0.602410,0.724638
0,poly svm,0.970000,0.806452,0.602410,0.689655
0,rand_poly_svm,0.978000,1.000000,0.602410,0.751880


# grid search in SVM

In [34]:

score_measure = "precision"
kfolds = 5

param_grid = {
    'C':[0.1,1,10,100],
    'gamma':[1,0.1,0.01,0.001],
    'kernel':['poly']
    
}

SVM_G_out = SVC()
grid_search = GridSearchCV(estimator = SVM_G_out, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 16 candidates, totalling 80 fits
The best precision score is 1.0
... with parameters: {'C': 0.1, 'gamma': 0.1, 'kernel': 'poly'}


In [35]:
model_preds = grid_search.predict(test_X)
c_matrix = confusion_matrix(test_y, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"grid_poly_svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,logistic using random & grid search,0.955333,0.586957,0.650602,0.617143
0,svm using Random & Grid search,0.955333,0.586957,0.650602,0.617143
0,svm using Random & Grid search,0.955333,0.586957,0.650602,0.617143
0,lin svm,0.978000,1.000000,0.602410,0.751880
0,rbf svm,0.974667,0.909091,0.602410,0.724638
0,poly svm,0.970000,0.806452,0.602410,0.689655
0,rand_poly_svm,0.978000,1.000000,0.602410,0.751880
0,grid_poly_svm,0.978000,1.000000,0.602410,0.751880


#  Decision Trees  using RandomSearch and GridSearch

In [41]:
#decision tree classifier
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(1,42),  
    'min_samples_leaf': np.arange(1,42),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 42), 
    'max_depth': np.arange(1,42), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(train_X,train_y)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is 0.6710359408033826
... with parameters: {'min_samples_split': 34, 'min_samples_leaf': 2, 'min_impurity_decrease': 0.0001, 'max_leaf_nodes': 39, 'max_depth': 21, 'criterion': 'entropy'}


C:\Users\santo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
45 fits failed out of a total of 2500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\santo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\santo\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "C:\Users\santo\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 250, in fit
    raise ValueError(
ValueError: min_samples_split must be an integer greater than 1 or a float in (

In [42]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(3,6),  
    'min_samples_leaf': np.arange(3,6),
    'min_impurity_decrease': np.arange(0.0009, 0.0012,0.0001),
    'max_leaf_nodes': np.arange(36,40), 
    'max_depth': np.arange(39,42), 
    'criterion': ['entropy'],
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X,train_y)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
The best recall score is 0.6893234672304439
... with parameters: {'criterion': 'entropy', 'max_depth': 39, 'max_leaf_nodes': 39, 'min_impurity_decrease': 0.001, 'min_samples_leaf': 3, 'min_samples_split': 5}


In [ ]:
From the analysis we can say that decision tree had good scope and the best fit among the models